In [4]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
from tqdm import tqdm
import collections
import random
import faiss
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

warnings.filterwarnings('ignore')

In [5]:
data_path = './data_raw/'
save_path = './temp_results/'
# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

metric_recall = False

In [6]:
# debug mode
def get_all_click_sample(data_path, sample_nums=10000):
    """
        subsample data for testing purposes
        data_path:
        sample_nums: sample size
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# get click data，online: train data+test data
# offlilne: train data only
def get_all_click_df(data_path='./data_raw/', offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = pd.concat([trn_click, tst_click], ignore_index=True)

    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [7]:
# reading basic item info
def get_item_info_df(data_path):
    item_info_df = pd.read_csv(data_path + 'articles.csv')
    
    item_info_df = item_info_df.rename(columns={'article_id': 'click_article_id'})
    
    return item_info_df

In [8]:
# reading article embedding data
def get_item_emb_dict(data_path):
    item_emb_df = pd.read_csv(data_path + 'articles_emb.csv')
    
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols])
    # normalization
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)

    item_emb_dict = dict(zip(item_emb_df['article_id'], item_emb_np))
    pickle.dump(item_emb_dict, open(save_path + 'item_content_emb.pkl', 'wb'))
    
    return item_emb_dict

In [9]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

In [10]:
# sample data
# all_click_df = get_all_click_sample(data_path)

# full data
all_click_df = get_all_click_df(offline=False)

# Normalize the click_timestamp column
all_click_df['click_timestamp'] = all_click_df[['click_timestamp']].apply(max_min_scaler)


In [11]:
item_info_df = get_item_info_df(data_path)

In [12]:
item_emb_dict = get_item_emb_dict(data_path)

In [13]:
#  creste a dict where each user maps to a dict of items and their clike times  {user1: {item1: time1, item2: time2..}...}
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')[['click_article_id', 'click_timestamp']].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

In [14]:
# create a dict where where each item maps to a dictionary of users and their click times:  {item1: {user1: time1, user2: time2...}...}
def get_item_user_time_dict(click_df):
    def make_user_time_pair(df):
        return list(zip(df['user_id'], df['click_timestamp']))
    
    click_df = click_df.sort_values('click_timestamp')
    item_user_time_df = click_df.groupby('click_article_id')[['user_id', 'click_timestamp']].apply(lambda x: make_user_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'user_time_list'})
    
    item_user_time_dict = dict(zip(item_user_time_df['click_article_id'], item_user_time_df['user_time_list']))
    return item_user_time_dict

In [15]:
# generate two dataframes: one contains all history clicks and the other one contains the last click for each user
def get_hist_and_last_click(all_click):
    
    all_click = all_click.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)

    #  If a user has only one click, their historical data will default to containing that single click to prevent the user from becoming "invisible" during training
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

In [16]:
# getting item info and save them as a dictionary
def get_item_info_dict(item_info_df):
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    item_info_df['created_at_ts'] = item_info_df[['created_at_ts']].apply(max_min_scaler)
    
    item_type_dict = dict(zip(item_info_df['click_article_id'], item_info_df['category_id']))
    item_words_dict = dict(zip(item_info_df['click_article_id'], item_info_df['words_count']))
    item_created_time_dict = dict(zip(item_info_df['click_article_id'], item_info_df['created_at_ts']))
    
    return item_type_dict, item_words_dict, item_created_time_dict

In [17]:
def get_user_hist_item_info_dict(all_click):
    
    # group items by their user_id and create a dictionary that maps each user to the set of category_id values they have interacted with
    user_hist_item_typs = all_click.groupby('user_id')['category_id'].agg(set).reset_index()
    user_hist_item_typs_dict = dict(zip(user_hist_item_typs['user_id'], user_hist_item_typs['category_id']))
    
    # create a dictionary that maps each user to the set of articles they clicked
    user_hist_item_ids_dict = all_click.groupby('user_id')['click_article_id'].agg(set).reset_index()
    user_hist_item_ids_dict = dict(zip(user_hist_item_ids_dict['user_id'], user_hist_item_ids_dict['click_article_id']))
    
    # create a dictionary that maps each user to average words count
    user_hist_item_words = all_click.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_hist_item_words_dict = dict(zip(user_hist_item_words['user_id'], user_hist_item_words['words_count']))
    
    # create a dictionary that maps each user to the his last clicktimestamp
    all_click_ = all_click.sort_values('click_timestamp')
    user_last_item_created_time = all_click_.groupby('user_id')['created_at_ts'].apply(lambda x: x.iloc[-1]).reset_index()
    
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    user_last_item_created_time['created_at_ts'] = user_last_item_created_time[['created_at_ts']].apply(max_min_scaler)
    
    user_last_item_created_time_dict = dict(zip(user_last_item_created_time['user_id'], \
                                                user_last_item_created_time['created_at_ts']))
    
    return user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict

In [18]:
# get items with top k clicks
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

## Define a multi-channel recall dictionary

In [19]:
#get article info
item_type_dict, item_words_dict, item_created_time_dict = get_item_info_dict(item_info_df)

In [103]:
#define a multi-channel recall dictionary to store the results of various recall methods
user_multi_recall_dict =  {'itemcf_sim_itemcf_recall': {},
                           'embedding_sim_item_recall': {},
                           'youtubednn_recall': {},
                           'cold_start_recall': {}}

In [21]:
# getting last one click for recall performance evaluation
trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)

## Metric to evaluation recall performance

In [22]:
# evaluate the hit rate at different cutoffs (e.g., top-10, top-20, top-30, top-40, and top-50 items)
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=5):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))
    user_num = len(user_recall_items_dict)
    
    for k in range(10, topk+1, 10):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            # get tok k items
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

## Computation of collaborative-filtering similarity matrix

In [23]:
#item to item similarity matrix
def itemcf_sim(df, item_created_time_dict):
    """
        item to item similarity
        :param df: dataset
        :item_created_time_dict:  
        return : item to item similarity matrix
        The function implements item-based collaborative filtering (ItemCF) enhanced with additional weights
    """
    
    user_item_time_dict = get_user_item_time(df)
    
    # 
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        for loc1, (i, i_click_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for loc2, (j, j_click_time) in enumerate(item_time_list):
                if(i == j):
                    continue
                    
                # Assigns higher importance to items clicked close together.
                loc_alpha = 1.0 if loc2 > loc1 else 0.7
                loc_weight = loc_alpha * (0.9 ** (np.abs(loc2 - loc1) - 1))
                # Penalizes pairs that were clicked far apart in time.
                click_time_weight = np.exp(0.7 ** np.abs(i_click_time - j_click_time))
                # Penalizes or rewards items based on how similar their creation times are.
                created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
                i2i_sim[i].setdefault(j, 0)
                # normalize to prevents users with long click sequences from disproportionately influencing the similarity score.
                i2i_sim[i][j] += loc_weight * click_time_weight * created_time_weight / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [24]:
i2i_sim = itemcf_sim(all_click_df, item_created_time_dict)

100%|█████████████████████████████████| 250000/250000 [01:43<00:00, 2419.74it/s]


In [25]:
# user to user similarity
def get_user_activate_degree_dict(all_click_df):
    all_click_df_ = all_click_df.groupby('user_id')['click_article_id'].count().reset_index()
    
    # normalization
    mm = MinMaxScaler()
    all_click_df_['click_article_id'] = mm.fit_transform(all_click_df_[['click_article_id']])
    user_activate_degree_dict = dict(zip(all_click_df_['user_id'], all_click_df_['click_article_id']))
    
    return user_activate_degree_dict

def usercf_sim(all_click_df, user_activate_degree_dict):
    """
        :param all_click_df: dataframe
        :param user_activate_degree_dict
        return user-based similarity matrix
        
        user-based collaborative filtering matrix computation
        
    """
    item_user_time_dict = get_item_user_time_dict(all_click_df)
    
    u2u_sim = {}
    user_cnt = defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        for u, click_time in user_time_list:
            user_cnt[u] += 1
            u2u_sim.setdefault(u, {})
            for v, click_time in user_time_list:
                u2u_sim[u].setdefault(v, 0)
                if u == v:
                    continue
                # activity weight
                activate_weight = 100 * 0.5 * (user_activate_degree_dict[u] + user_activate_degree_dict[v])   
                u2u_sim[u][v] += activate_weight / math.log(len(user_time_list) + 1)
    
    u2u_sim_ = u2u_sim.copy()
    for u, related_users in u2u_sim.items():
        for v, wij in related_users.items():
            u2u_sim_[u][v] = wij / math.sqrt(user_cnt[u] * user_cnt[v])
    
    # save to local file
    pickle.dump(u2u_sim_, open(save_path + 'usercf_u2u_sim.pkl', 'wb'))

    return u2u_sim_

In [27]:
# too much storage required to compute u2u_sim so halt the task here
# run on sample data
#user_activate_degree_dict = get_user_activate_degree_dict(all_click_df)
#u2u_sim = usercf_sim(all_click_df, user_activate_degree_dict)

In [28]:
# Item embedding similarity using Faiss
# Faiss search returns the top k item based on similarty score for each item
def embdding_sim(click_df, item_emb_df, save_path, topk):
    """
        Item similarity calculation based on content-based embedding
        :param click_df: dataframe
        :param item_emb_df: embedding of articles
        :param save_path: save path
        :patam topk: top k most similar
        return dictionary
        
    """
    
    # mapping item index to raw id 
    item_idx_2_rawid_dict = dict(zip(item_emb_df.index, item_emb_df['article_id']))
    
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols].values, dtype=np.float32)
    # vector normilization
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)
    
    # construct faiss index
    item_index = faiss.IndexFlatIP(item_emb_np.shape[1])
    item_index.add(item_emb_np)
    # similarity search，return top k items and respective similarity score for every index
    sim, idx = item_index.search(item_emb_np, topk) # return a list
    
    # saving search results with corresponding raw ID
    item_sim_dict = collections.defaultdict(dict)
    for target_idx, sim_value_list, rele_idx_list in tqdm(zip(range(len(item_emb_np)), sim, idx)):
        target_raw_id = item_idx_2_rawid_dict[target_idx]
        # starting from index 1 to exclude the item itself
        for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]): 
            rele_raw_id = item_idx_2_rawid_dict[rele_idx]
            item_sim_dict[target_raw_id][rele_raw_id] = item_sim_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value
    
    # save i2i similarity matrix
    pickle.dump(item_sim_dict, open(save_path + 'emb_i2i_sim.pkl', 'wb'))   
    
    return item_sim_dict

In [30]:
item_emb_df = pd.read_csv(data_path + '/articles_emb.csv')
emb_i2i_sim = embdding_sim(all_click_df, item_emb_df, save_path, topk=10) 

364047it [00:04, 87063.15it/s]


# Recall generation using item to item similarity

In [31]:
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim):
    """
        recall based on item CF
        :param user_id: user id
        :param user_item_time_dict: dict   {user1: {item1: time1, item2: time2..}...}
        :param i2i_sim: dict 
        :param sim_item_topk: int
        :param recall_item_num: int
        :param item_topk_click: list of most popular articles to fill the individual user's recalled list in case there is vacancy
        :param emb_i2i_sim: dict
        
        return: recall article dict {item1:score1, item2: score2...}
        
    """
    # get user historical interacted items
    user_hist_items = user_item_time_dict[user_id]
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items:
                continue
            
            # weight for difference in article cerated time
            created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            # Applies a positional weight based on how recent the interaction with item i is
            loc_weight = (0.9 ** (len(user_hist_items) - loc))
            
            content_weight = 1.0
            if emb_i2i_sim.get(i, {}).get(j, None) is not None:
                content_weight += emb_i2i_sim[i][j]
            if emb_i2i_sim.get(j, {}).get(i, None) is not None:
                content_weight += emb_i2i_sim[j][i]
                
            item_rank.setdefault(j, 0)
            item_rank[j] += created_time_weight * loc_weight * content_weight * wij
    
    # fill the item list with popular items if fewer than 10
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # item to be filled should not be in the original list
                continue
            item_rank[item] = - i - 100 # any negative number
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

## Recall using both i2iCF and item embedding

In [32]:
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)

i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))
emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))

sim_item_topk = 20
recall_item_num = 10
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        item_topk_click, item_created_time_dict, emb_i2i_sim)

user_multi_recall_dict['itemcf_sim_itemcf_recall'] = user_recall_items_dict
pickle.dump(user_multi_recall_dict['itemcf_sim_itemcf_recall'], open(save_path + 'itemcf_recall_dict.pkl', 'wb'))

if metric_recall:
    #
    metrics_recall(user_multi_recall_dict['itemcf_sim_itemcf_recall'], trn_last_click_df, topk=recall_item_num)

100%|██████████████████████████████████| 250000/250000 [29:17<00:00, 142.21it/s]


## Recall generation using item embedding only


In [33]:
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df#extract last click for evaluation

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)
i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl','rb'))

sim_item_topk = 20
recall_item_num = 10

item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, sim_item_topk, 
                                                        recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim)
    
user_multi_recall_dict['embedding_sim_item_recall'] = user_recall_items_dict#saving into the multi recall dictionary
pickle.dump(user_multi_recall_dict['embedding_sim_item_recall'], open(save_path + 'embedding_sim_item_recall.pkl', 'wb'))

if metric_recall:
    # recall performance evaluation
    metrics_recall(user_multi_recall_dict['embedding_sim_item_recall'], trn_last_click_df, topk=recall_item_num)

100%|█████████████████████████████████| 250000/250000 [00:32<00:00, 7800.57it/s]


# Recall Generation using YouTube DNN, aka Two Tower

In [34]:
# preparing data for training two-tower model

def gen_data_set(data, negsample=0):
    data.sort_values("click_timestamp", inplace=True)
    item_ids = data['click_article_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['click_article_id'].tolist()
        
        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))   # select neg sample from unread articles
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)  # select n negsamples for every positive sample
            
        # For users with only one interaction, add that single interaction to both the training and test sets to prevent missing embeddings during training.
        if len(pos_list) == 1:
            train_set.append((reviewerID, [pos_list[0]], pos_list[0],1,len(pos_list)))
            test_set.append((reviewerID, [pos_list[0]], pos_list[0],1,len(pos_list)))
            
        # construct sample using sliding window
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1])))  # 正样本 [user_id, his_item, pos_item, label, len(his_item)]
                for negi in range(negsample):
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1]))) # 负样本 [user_id, his_item, neg_item, label, len(his_item)]
            else:
                # making the longest sequence as test data
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1])))
                
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    return train_set, test_set

# padding intput data to ensure consistent input sizes
def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "click_article_id": train_iid, "hist_article_id": train_seq_pad,
                         "hist_len": train_hist_len}

    return train_model_input, train_label

## Define model architecture

In [83]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, GRU, Flatten, Concatenate

# Input layers
user_id_input = Input(shape=(1,), name="user_id")
click_article_id_input = Input(shape=(1,), name="click_article_id")
hist_article_id_input = Input(shape=(10,), name="hist_article_id")  # Example max_len=10
hist_len_input = Input(shape=(1,), name="hist_len")

# Embedding layers
embedding_dim = 16
user_embedding = Embedding(input_dim=250000, output_dim=embedding_dim, name="user_embedding")(user_id_input)
item_embedding = Embedding(input_dim=35380, output_dim=embedding_dim, name="item_embedding")(click_article_id_input)
hist_embedding = Embedding(input_dim=35380, output_dim=embedding_dim, name="hist_embedding")(hist_article_id_input)

# GRU for sequence modeling
gru_out = GRU(units=16, return_sequences=False)(hist_embedding)

# Combine embeddings
concat = Concatenate()([Flatten()(user_embedding), Flatten()(item_embedding), gru_out])

# Dense layers for prediction
dense_out = Dense(units=64, activation="relu")(concat)
output = Dense(units=1, activation="sigmoid")(dense_out)

# Build model
model = Model(inputs={"user_id":user_id_input,"click_article_id":click_article_id_input,"hist_article_id":hist_article_id_input,"hist_len": hist_len_input},outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Formatting input data

In [84]:
# split train data and test data
SEQ_LEN = 10 # length of the user click sequence
user_profile_ = all_click_df[["user_id"]].drop_duplicates('user_id')
item_profile_ = all_click_df[["click_article_id"]].drop_duplicates('click_article_id')  
# Catogorical encoding
features = ["click_article_id", "user_id"]
feature_max_idx = {}
    
for feature in features:
    lbe = LabelEncoder()
    all_click_df[feature] = lbe.fit_transform(all_click_df[feature])
    feature_max_idx[feature] = all_click_df[feature].max() + 1
    

# extract user and item profile
user_profile = all_click_df[["user_id"]].drop_duplicates('user_id')
item_profile = all_click_df[["click_article_id"]].drop_duplicates('click_article_id')  
user_index_2_rawid = dict(zip(user_profile['user_id'], user_profile_['user_id']))
item_index_2_rawid = dict(zip(item_profile['click_article_id'], item_profile_['click_article_id']))
 
train_set, test_set = gen_data_set(all_click_df, 0)
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
X_train = {
    "user_id": train_model_input["user_id"],
    "click_article_id": train_model_input["click_article_id"],
    "hist_article_id": train_model_input["hist_article_id"],
    "hist_len": train_model_input["hist_len"]
}
y_train = train_label.reshape((-1, 1))

100%|████████████████████████████████| 250000/250000 [00:08<00:00, 30124.61it/s]


In [85]:
for key, value in X_train.items():
    print(f"Input {key}: {value.shape}")
print(all_click_df["click_article_id"].nunique())
print("Concatenated shape:", concat.shape)  # Expected: (batch_size, 48)
print(f"Shape of hist_article_id: {y_train.shape}")


Input user_id: (1149673,)
Input click_article_id: (1149673,)
Input hist_article_id: (1149673, 10)
Input hist_len: (1149673,)
35380
Concatenated shape: (None, 48)
Shape of hist_article_id: (1149673, 1)


In [87]:
# train the model
model.fit(X_train, y_train, batch_size=128, epochs=1, validation_split=0.2)


7186/7186 ━━━━━━━━━━━━━━━━━━━━ 71s 10ms/step - accuracy: 0.9992 - loss: 0.0157 - val_accuracy: 1.0000 - val_loss: 8.4651e-08


In [88]:
# retrieve embedding layers
user_embedding_layer = model.get_layer("user_embedding")
item_embedding_layer = model.get_layer("item_embedding")


In [89]:
# retrieve the embedding weights
# User embeddings
user_embs = user_embedding_layer.get_weights()[0]

# Item embeddings
item_embs = item_embedding_layer.get_weights()[0]

print("User embedding matrix shape:", user_embs.shape)
print("Item embedding matrix shape:", item_embs.shape)
#test_user_model_input = test_model_input
#all_item_model_input = {"click_article_id": item_profile['click_article_id'].values}
# normalization before saving to local file
user_embs = user_embs / np.linalg.norm(user_embs, axis=1, keepdims=True)
item_embs = item_embs / np.linalg.norm(item_embs, axis=1, keepdims=True)
 # convert Embedding to dict for fast reference
raw_user_id_emb_dict = {user_index_2_rawid[k]: \
                                v for k, v in zip(user_profile['user_id'], user_embs)}
raw_item_id_emb_dict = {item_index_2_rawid[k]: \
                                v for k, v in zip(item_profile['click_article_id'], item_embs)}
# save Embedding to local
pickle.dump(raw_user_id_emb_dict, open(save_path + 'user_youtube_emb.pkl', 'wb'))
pickle.dump(raw_item_id_emb_dict, open(save_path + 'item_youtube_emb.pkl', 'wb'))
        
len(item_index_2_rawid)

User embedding matrix shape: (250000, 16)
Item embedding matrix shape: (35380, 16)


35380

In [90]:
# Batch candidate generation, using Approximate Nearest Neighbours for efficient search
# Build a FAISS index
embedding_dim = item_embs.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Inner product similarity
faiss_index.add(item_embs)          # Add item embeddings

# Perform batch search for top-k items
batch_size = user_embs.shape[0]
top_k = 10
sim, idx = faiss_index.search(user_embs, top_k)

In [91]:
idx[1,:]

array([35155, 22393, 21950,  1695, 22980,  5818, 28566, 30313, 22728,
       25791])

In [95]:
user_recall_items_dict = collections.defaultdict(dict)
for target_idx, sim_value_list, rele_idx_list in tqdm(zip(test_model_input['user_id'], sim, idx)):
    target_raw_id = user_index_2_rawid[target_idx]
    #Iterates through the top-k recommended items (excluding the item itself).
    for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]): 
        rele_raw_id = item_index_2_rawid[rele_idx]
        user_recall_items_dict[target_raw_id][rele_raw_id] = user_recall_items_dict.get(target_raw_id, {})\
                                                                    .get(rele_raw_id, 0) + sim_value
            
user_recall_items_dict = {k: sorted(v.items(), key=lambda x: x[1], reverse=True) for k, v in user_recall_items_dict.items()}
# ranking recalled items

pickle.dump(user_recall_items_dict, open(save_path + 'youtube_u2i_dict.pkl', 'wb'))


In [94]:
for target_idx, _, rele_idx_list in zip(test_model_input['user_id'], sim, idx):
    for rele_idx in rele_idx_list:
        if rele_idx not in item_index_2_rawid:
            print(f"Missing index: {rele_idx}")


In [96]:
user_multi_recall_dict['youtubednn_recall'] = user_recall_items_dict

## Cold start recall

In [97]:

trn_hist_click_df = all_click_df

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)
i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl','rb'))

sim_item_topk = 150
recall_item_num = 100 # 稍微召回多一点文章，便于后续的规则筛选

item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)
for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, sim_item_topk, 
                                                        recall_item_num, item_topk_click,item_created_time_dict, emb_i2i_sim)
pickle.dump(user_recall_items_dict, open(save_path + 'cold_start_items_raw_dict.pkl', 'wb'))

100%|█████████████████████████████████| 250000/250000 [00:42<00:00, 5823.94it/s]


In [139]:

def get_click_article_ids_set(all_click_df):
    return set(all_click_df.click_article_id.values)

def cold_start_items(user_recall_items_dict, user_hist_item_typs_dict, user_hist_item_words_dict, \
                     user_last_item_created_time_dict, item_type_dict, item_words_dict, 
                     item_created_time_dict, click_article_ids_set, recall_item_num):
    """
        冷启动的情况下召回一些文章
        :param user_recall_items_dict: 基于内容embedding相似性召回来的很多文章， 字典， {user1: [item1, item2, ..], }
        :param user_hist_item_typs_dict: 字典， 用户点击的文章的主题映射
        :param user_hist_item_words_dict: 字典， 用户点击的历史文章的字数映射
        :param user_last_item_created_time_idct: 字典，用户点击的历史文章创建时间映射
        :param item_tpye_idct: 字典，文章主题映射
        :param item_words_dict: 字典，文章字数映射
        :param item_created_time_dict: 字典， 文章创建时间映射
        :param click_article_ids_set: 集合，用户点击过得文章, 也就是日志里面出现过的文章
        :param recall_item_num: 召回文章的数量， 这个指的是没有出现在日志里面的文章数量
    """
    
    cold_start_user_items_dict = {}
    for user, item_list in tqdm(user_recall_items_dict.items()):
        cold_start_user_items_dict.setdefault(user, [])
        for item, score in item_list:
            # getting historical data
            hist_item_type_set = user_hist_item_typs_dict[user]
            hist_mean_words = user_hist_item_words_dict[user]
            hist_last_item_created_time = user_last_item_created_time_dict[user]
            hist_last_item_created_time = datetime.fromtimestamp(hist_last_item_created_time)
            
            # getting the info of current recalled article
            curr_item_type = item_type_dict[item]
            curr_item_words = item_words_dict[item]
            curr_item_created_time = item_created_time_dict[item]
            curr_item_created_time = datetime.fromtimestamp(curr_item_created_time)

            # 首先，文章不能出现在用户的历史点击中， 然后根据文章主题，文章单词数，文章创建时间进行筛选
            if curr_item_type not in hist_item_type_set or \
                item in click_article_ids_set or \
                abs(curr_item_words - hist_mean_words) > 200 or \
                abs((curr_item_created_time - hist_last_item_created_time).days) > 90: 
                continue
                
            cold_start_user_items_dict[user].append((item, score))      # {user1: [(item1, score1), (item2, score2)..]...}
        print(f"Processed {user} - {len(cold_start_user_items_dict[user])} items recommended.")

    # 需要控制一下冷启动召回的数量
    cold_start_user_items_dict = {k: sorted(v, key=lambda x:x[1], reverse=True)[:recall_item_num] \
                                  for k, v in cold_start_user_items_dict.items()}
    
    pickle.dump(cold_start_user_items_dict, open(save_path + 'cold_start_user_items_dict.pkl', 'wb'))
    
    return cold_start_user_items_dict

In [140]:
all_click_df_ = all_click_df.copy()
all_click_df_ = all_click_df_.merge(item_info_df, how='left', on='click_article_id')
user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict = get_user_hist_item_info_dict(all_click_df_)
click_article_ids_set = get_click_article_ids_set(all_click_df)

cold_start_user_items_dict = cold_start_items(user_recall_items_dict, user_hist_item_typs_dict, user_hist_item_words_dict, \
                                              user_last_item_created_time_dict, item_type_dict, item_words_dict, \
                                              item_created_time_dict, click_article_ids_set, recall_item_num)

user_multi_recall_dict['cold_start_recall'] = cold_start_user_items_dict

# Combine recall results from multiple ways

In [120]:
def combine_recall_results(user_multi_recall_dict, weight_dict=None, topk=25):
    final_recall_items_dict = {}
    
    # normalizing user recall articles for easy aggregation
    def norm_user_recall_items_sim(sorted_item_list):
        # 如果冷启动中没有文章或者只有一篇文章，直接返回，出现这种情况的原因可能是冷启动召回的文章数量太少了，
        # 基于规则筛选之后就没有文章了, 这里还可以做一些其他的策略性的筛选
        if len(sorted_item_list) < 2:
            return sorted_item_list
        
        min_sim = sorted_item_list[-1][1]
        max_sim = sorted_item_list[0][1]
        
        norm_sorted_item_list = []
        for item, score in sorted_item_list:
            if max_sim > 0:
                norm_score = 1.0 * (score - min_sim) / (max_sim - min_sim) if max_sim > min_sim else 1.0
            else:
                norm_score = 0.0
            norm_sorted_item_list.append((item, norm_score))
            
        return norm_sorted_item_list
    
    print('multi recall merging...')
    for method, user_recall_items in tqdm(user_multi_recall_dict.items()):
        print(method + '...')
        # set a weight to result from each recall method
        if weight_dict == None:
            recall_method_weight = 1
        else:
            recall_method_weight = weight_dict[method]
        
        for user_id, sorted_item_list in user_recall_items.items(): # 进行归一化
            user_recall_items[user_id] = norm_user_recall_items_sim(sorted_item_list)
        
        for user_id, sorted_item_list in user_recall_items.items():
            # print('user_id')
            final_recall_items_dict.setdefault(user_id, {})
            for item, score in sorted_item_list:
                final_recall_items_dict[user_id].setdefault(item, 0)
                final_recall_items_dict[user_id][item] += recall_method_weight * score  
    
    final_recall_items_dict_rank = {}
    for user, recall_item_dict in final_recall_items_dict.items():
        final_recall_items_dict_rank[user] = sorted(recall_item_dict.items(), key=lambda x: x[1], reverse=True)[:topk]

    # saving to local
    pickle.dump(final_recall_items_dict, open(os.path.join(save_path, 'final_recall_items_dict.pkl'),'wb'))

    return final_recall_items_dict_rank

In [121]:
# set weights for similarity scores computed from multiple methods
weight_dict = {'itemcf_sim_itemcf_recall': 1.0,
               'embedding_sim_item_recall': 1.0,
               'youtubednn_recall': 1.0,
               'cold_start_recall': 1.0}

In [141]:
# ranking the top k recalled items
final_recall_items_dict_rank = combine_recall_results(user_multi_recall_dict, topk=100)

AttributeError: 'dict' object has no attribute 'info'

In [138]:
print(len(user_recall_items_dict))